<a href="https://colab.research.google.com/github/Gerviba/deeplearning-human-posture-recogn/blob/master/transformer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from PIL import Image
import numpy as np
import os

In [ ]:
AGENT = "A01"
FIGS = ["F1", "F2", "F3", "F4", "F5", "F6"]
FRAMES = [["1", "2", "3", "4", "5"], 
          ["1", "2", "3", "4", "5"],
          ["1", "2", "3", "4"],
          ["1", "2", "3", "4"],
          ["1", "2", "3", "4", "5"],
          ["1", "2", "3"]]

FIGS_AYNC = ["F7", "F8", "F9", "F10", "F11", "F12", "F13"]
FRAMES_ASYNC = [["1", "2", "3"],
                ["1", "2", "3"],
                ["1", "2", "3"],
                ["1", "2", "3"],
                ["1", "2", "3"],
                ["1", "2", "3"],
                ["1", "2", "3"]]

# Input file: input/{AGENT}_{FIG}_{FRAME}.jpg
# Output file format (generated async image): output/L{L-FIG}-{L-FRAME}_R{R-FIG}-{R-FRAME}/L{L-FIG}-{L-FRAME}_R{R-FIG}-{R-FRAME}_{AGENT}.jpg
# Output file format (simple async image): output/{FIG}-{FRAME}/{FIG}-{FRAME}_{AGENT}.jpg


---


In [ ]:
fig = FIGS[0]
frame = FRAMES[0][0]

leftHalves = np.empty((len(FIGS), 5, 563, 500, 3))
rightHalves = np.empty((len(FIGS), 5, 563, 500, 3))

TARGET_WIDTH = 500

figId = 0
for fig in FIGS:
  frameId = 0

  for frame in FRAMES[figId]:
    im = np.array(Image.open("input/%s_%s_%s.jpg" % (AGENT, fig, frame)))
    im = im[::4, ::4, :]
    print(im.shape)

    leftHalves[figId, frameId] = (im[:, :500, :].copy())
    rightHalves[figId, frameId] = (im[:, 500:, :].copy())
    
    frameId += 1
  figId += 1


In [ ]:
figLId = 0
for figL in FIGS:
  frameLId = 0
  for frameL in FRAMES[figLId]:
    figRId = 0
    for figR in FIGS:
      frameRId = 0
      for frameR in FRAMES[figRId]:
        imL = leftHalves[figLId][frameLId]
        imR = rightHalves[figRId][frameRId]

        im = np.concatenate((imL, imR), axis=1)
        im = im.astype(np.uint8)

        pil_img = Image.fromarray(im)
        dir = "output/L%s-%s_R%s-%s" % (figL, frameL, figR, frameR)
        if not os.path.exists(dir):
          os.makedirs(dir)
        fileName = "%s/L%s-%s_R%s-%s_%s.jpg" % (dir, figL, frameL, figR, frameR, AGENT)
        pil_img.save(fileName)
        print("Saved: %s" % (fileName))
        frameRId += 1
      figRId += 1
    frameLId += 1
  figLId += 1

### Async images read 

In [ ]:
pics = np.empty((len(FIGS_AYNC), 5, 563, 500, 3))

for fig_idx, fig in enumerate(FIGS_AYNC):
  for frame_idx, frame in enumerate(FRAMES_ASYNC[fig_idx]):
    im = np.array(Image.open("input/%s_%s_%s" % (AGENT, fig, frame)))
    im = im[::4, ::4, :]
    print(im.shape)
    pics[fig_idx, frame_idx] = (im[:, :, :].copy())

### Async images write

In [ ]:
for fig_idx, fig in enumerate(FIGS_AYNC):
  for frame_idx, frame in enumerate(FRAMES_ASYNC[fig_idx]):
    im = pics[fig_idx][frame_idx]
    im = im.astype(np.uint8)
    pil_img = Image.fromarray(im)
    fig_frame = "%s-%s" % (fig, frame)
    # output/FIG-FRAME/FIG-FRAME_AGENT.jpg
    dir = "output/%s" % (fig_frame)

    if not os.path.exists(dir):
      os.makedirs(dir)

    fileName = "%s/%s_%s.jpg" % (dir, fig_frame, AGENT)
    pil_img.save(fileName)
    print("Saved: %s" %(fileName))
    

In [ ]:
!rm -r input/*.jpg